# TorchScript 
TorchScript is a way to create serializable and optimizable models from PyTorch code. Any TorchScript program can be saved from a Python process and loaded in a process where there is no Python dependency.

It is a tool to incrementally transition a model from a pure Python program to a TorchScript program that can be run independently from Python, such as in a standalone C++ program. This makes it possible to train models in PyTorch using familiar tools in Python and then export the model via TorchScript to a production environment where Python programs may be disadvantageous for performance and multi-threading reasons.

In [1]:
from torchvision import models
import cv2
import torch
from torchvision.transforms import Resize, Compose, ToTensor, Normalize
import onnx
from torch import nn

In [2]:
def preprocess_image(img_path):
    # transformations for the input data
    transforms = Compose([
        ToTensor(),
        Resize(224),
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    # read input image
    input_img = cv2.imread(img_path)
    # do transformations
    input_data = transforms(input_img)
    batch_data = torch.unsqueeze(input_data, 0)
    return batch_data

def postprocess(output_data):
    # get class names
    with open("../data/imagenet_classes.txt") as f:
        classes = [line.strip() for line in f.readlines()]
    # calculate human-readable value by softmax
    confidences = torch.nn.functional.softmax(output_data, dim=1)[0] * 100
    # find top predicted classes
    _, indices = torch.sort(output_data, descending=True)
    i = 0
    # print the top classes predicted by the model
    while confidences[indices[0][i]] > 0.5:
        class_idx = indices[0][i]
        print(
            "class:",
            classes[class_idx],
            ", confidence:",
            confidences[class_idx].item(),
            "%, index:",
            class_idx.item(),
        )
        i += 1


### Step 1 : Simple Baseline Python implementation

In [3]:
input = preprocess_image("../data/turkish_coffee.jpg").cuda()
model = models.resnet50(pretrained=True)
model.eval()
model.cuda()
output = model(input)

postprocess(output)

class: cup , confidence: 94.97858428955078 %, index: 968
class: espresso , confidence: 3.9512522220611572 %, index: 967
class: coffee mug , confidence: 0.6196928024291992 %, index: 504


### Step 2 : Define nn.Module based network class

In [4]:
class Classify(nn.Module):
    def __init__(self):
        super(Classify, self).__init__()
        self.resnet = models.resnet50(pretrained=True)
        
    def forward(self, image):
        output = self.resnet(image)
        return output
   

In [5]:
network = Classify().cuda()

###  Step 3 : Convert network to Trace using dummy input

In [7]:
dummy_input = torch.rand_like(input).cuda()
traced_network = torch.jit.trace(network, (dummy_input))

### Step 4 : Save the Trace as a Torchscript file

In [8]:
traced_network.save('../torchscript_engines/simple_classifier.pt')

To load your serialized PyTorch model in C++, your application must depend on the PyTorch C++ API – also known as LibTorch. The LibTorch distribution encompasses a collection of shared libraries, header files and CMake build configuration files.  For this tutorial, we will be building a minimal C++ application using setup-tools and LibTorch that simply loads and executes a serialized PyTorch model.